In [1]:
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
import torch.optim as optim

import torchdiffeq

from tensorboard_utils import Tensorboard
from tensorboard_utils import tensorboard_event_accumulator

import transformer.Constants as Constants
from transformer.Layers import EncoderLayer, DecoderLayer
from transformer.Modules import ScaledDotProductAttention
from transformer.Models import Decoder, get_attn_key_pad_mask, get_non_pad_mask, get_sinusoid_encoding_table
from transformer.SubLayers import PositionwiseFeedForward

import dataset

import model_process
import checkpoints
from node_transformer import NodeTransformer

import matplotlib
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib notebook  
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

print("Torch Version", torch.__version__)

%load_ext autoreload
%autoreload 2

Torch Version 1.1.0


In [2]:
seed = 1
torch.manual_seed(seed)
device = torch.device("cuda")
print("device", device)

device cuda


In [3]:
data = torch.load("/home/mandubian/datasets/multi30k/multi30k.atok.low.pt")

In [4]:
max_token_seq_len = data['settings'].max_token_seq_len
print(max_token_seq_len)

52


In [5]:
train_loader, val_loader = dataset.prepare_dataloaders(data, batch_size=128)

### Create an experiment with a name and a unique ID

In [6]:
exp_name = "transformer_multi30k"
unique_id = "2019-06-07_1000"


### Create Model

In [7]:
model = None

In [8]:
src_vocab_sz = train_loader.dataset.src_vocab_size
print("src_vocab_sz", src_vocab_sz)
tgt_vocab_sz = train_loader.dataset.tgt_vocab_size
print("tgt_vocab_sz", tgt_vocab_sz)

if model:
    del model
    
model = NodeTransformer(
    n_src_vocab=max(src_vocab_sz, tgt_vocab_sz),
    n_tgt_vocab=max(src_vocab_sz, tgt_vocab_sz),
    len_max_seq=max_token_seq_len,
    #emb_src_tgt_weight_sharing=False,
    #d_word_vec=128, d_model=128, d_inner=512,
    n_head=8, method='dopri5-ext', rtol=1e-3, atol=1e-3,
    has_node_encoder=False, has_node_decoder=False)

model = model.to(device)

src_vocab_sz 9795
tgt_vocab_sz 17989


### Create Tensorboard metrics logger

In [9]:
tb = Tensorboard(exp_name, unique_name=unique_id)

Writing TensorBoard events locally to runs/transformer_multi30k_2019-06-07_1000


### Create basic optimizer

In [11]:
optimizer = optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.995), eps=1e-9)


### Train

In [12]:
# Continuous space discretization
timesteps = np.linspace(0., 1, num=6)
timesteps = torch.from_numpy(timesteps).float()

EPOCHS = 50
LOG_INTERVAL = 5

#from torch import autograd
#with autograd.detect_anomaly():
model_process.train(
    exp_name, unique_id,
    model, 
    train_loader, val_loader, timesteps,
    optimizer, device,
    epochs=EPOCHS, tb=tb, log_interval=LOG_INTERVAL,
    #start_epoch=0, best_valid_accu=state["acc"]
)

Loaded model and timesteps to cuda


[ Epoch 0 ]
Adding group train to writers (dict_keys([]))
[Training]  loss: 6.8149000610367345, ppl:  911.32543, accuracy: 27.076 %, elapse: 19205.983ms
Adding group eval to writers (dict_keys(['train']))
[Validation]  loss: 5.35197686920177,  ppl:  211.02505, accuracy: 34.644 %, elapse: 285.239ms
Checkpointing Validation Model...


[ Epoch 1 ]
[Training]  loss: 5.709207316331694, ppl:  301.63187, accuracy: 37.813 %, elapse: 19029.006ms
[Validation]  loss: 4.641788004155026,  ppl:  103.72965, accuracy: 41.214 %, elapse: 285.800ms
Checkpointing Validation Model...


[ Epoch 2 ]
[Training]  loss: 5.221279466281199, ppl:  185.17095, accuracy: 42.742 %, elapse: 19098.102ms
[Validation]  loss: 4.201269888042661,  ppl:  66.77107, accuracy: 45.018 %, elapse: 287.158ms
Checkpointing Validation Model...


[ Epoch 3 ]
[Training]  loss: 4.882612129451231, ppl:  131.97495, accuracy: 46.122 %, elapse: 19079.331ms
[Validation]  loss: 3.87762375528333,  ppl:  48.30928, accuracy: 47.912 %, elapse: 286.661ms
Checkpointing Validation Model...


...

[ Epoch 45 ]
[Training]  loss: 2.4313040332251825, ppl:  11.37370, accuracy: 78.162 %, elapse: 19217.146ms
[Validation]  loss: 2.589681234441042,  ppl:  13.32552, accuracy: 61.272 %, elapse: 286.766ms


[ Epoch 46 ]
[Training]  loss: 2.407496511081418, ppl:  11.10612, accuracy: 78.595 %, elapse: 19273.083ms
[Validation]  loss: 2.5865377686718696,  ppl:  13.28370, accuracy: 61.172 %, elapse: 287.521ms


[ Epoch 47 ]
[Training]  loss: 2.387569536504241, ppl:  10.88700, accuracy: 78.908 %, elapse: 19187.665ms
[Validation]  loss: 2.588779196133543,  ppl:  13.31351, accuracy: 61.537 %, elapse: 287.255ms


[ Epoch 48 ]
[Training]  loss: 2.3634617543730814, ppl:  10.62768, accuracy: 79.446 %, elapse: 19346.606ms
[Validation]  loss: 2.586259278008364,  ppl:  13.28000, accuracy: 61.244 %, elapse: 287.271ms


[ Epoch 49 ]
[Training]  loss: 2.3470888432062367, ppl:  10.45509, accuracy: 79.760 %, elapse: 19267.039ms
[Validation]  loss: 2.578025202994036,  ppl:  13.17110, accuracy: 61.459 %, elapse: 287.232ms



### Restore best checkpoint (to restart past training)

In [ ]:
state = checkpoints.restore_best_checkpoint(
    "node_transformer_multi30k", "2019-05-29_1200", "validation", model, optimizer)

print("accuracy", state["acc"])
print("loss", state["loss"])
model = model.to(device)